In [1]:
import pandas as pd 
import numpy as np 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import wordnet as wn 
from sklearn import model_selection,naive_bayes,svm
from sklearn.metrics import accuracy_score
import re
import nltk
from sklearn.svm import SVC
from sklearn import metrics
# utility function for text cleaning and metrics 
#from utils import *
from math import log
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [2]:
df=pd.read_csv('../dataset/founta.csv')

In [3]:
df.dropna(inplace=True)

In [4]:
#shuffle the dataset
#from sklearn.utils import shuffle
#df = shuffle(df)

In [5]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

In [6]:
import re

text = ' '.join(df['text'])
text = text.split()
freq_comm = pd.Series(text).value_counts()
rare = freq_comm[freq_comm.values == 1]

def get_clean_text(x):
    if type(x) is str:
        x = x.lower()
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        x = re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x) 
        #regex to remove to emails
        x = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x)
        #regex to remove URLs
        x = re.sub('RT', "", x)
        #substitute the 'RT' retweet tags with empty spaces
        x = re.sub('[^A-Z a-z]+', '', x)
        #combining all the text excluding rare words.
        x = ' '.join([t for t in x.split() if t not in rare])
        return x
    else:
        return x
    
df['text'] = df['text'].apply(lambda x: get_clean_text(x)) 

In [7]:
text = df['text'].tolist()

In [8]:
y = df['label']

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(text,y,test_size=0.1,random_state=11)

In [10]:
#label encoder
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

In [11]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['text'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [12]:
#model = svm.SVC(C=1, kernel='linear', gamma='auto',probability=True)
#model = RandomForestClassifier(max_depth=10, random_state=0)
model = LogisticRegression(random_state=0)
model.fit(Train_X_Tfidf,y_train)
# predict the labels on validation dataset
predictions = model.predict(Test_X_Tfidf)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      1111
           1       1.00      0.26      0.42        76

    accuracy                           0.95      1187
   macro avg       0.98      0.63      0.70      1187
weighted avg       0.96      0.95      0.94      1187



In [13]:
score=geometric_mean_score(y_test, predictions)

In [14]:
score

0.512989176042577

In [15]:
f1_score(y_test,predictions)

0.4166666666666667

In [16]:
print(accuracy_score(y_test,predictions))

0.9528222409435552


In [17]:
#y_pred_keras = model.predict(X_test)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, predictions)

In [18]:
auc_model = auc(fpr_keras, tpr_keras)

In [19]:
auc_model

0.631578947368421

In [20]:
result=pd.DataFrame({'text':X_test,'label':predictions})

In [21]:
result.to_csv("results_tfidf_lr.csv")